In [1]:
import pandas as pd
import re
corpus_lemm = pd.read_csv("reviews_score.csv", engine='python')

In [3]:
text = '.'.join(corpus_lemm['review'])
sentences_list = [x.strip() for x in text.split('.')]

### Идея 2

In [4]:
words_sentences_list = [x.split(' ') for x in sentences_list]

In [8]:
from pymystem3 import Mystem
m = Mystem()
def preprocessing(raw_text):
    clean_text = re.sub('\W+', ' ', raw_text) # \W = [^a-zA-Z0-9_]
    return clean_text

for i in range(len(words_sentences_list)):
    for j in range(len(words_sentences_list[i])):
        words_sentences_list[i][j] = preprocessing(words_sentences_list[i][j])

In [9]:
words_sentences_list = [word for word in words_sentences_list if word != ' ']

In [10]:
import gensim
model = gensim.models.Word2Vec(words_sentences_list, size=500, window=10, min_count=2, sg=0)

In [11]:
model

In [12]:
import pickle
pickle.dump(model, open("w2v_model", "wb"))
#model = pickle.load(open("w2v_model", "rb"))

In [13]:
model.wv.save_word2vec_format('model.bin', binary=True)

## пересечение со словарем

In [2]:
with open('sentiment_list.txt','r',encoding='utf-8') as file4:
    new_sent_words = {word.split('\t')[0].strip() for word in file4.readlines()}

In [3]:
with open('rusentilex.txt','r',encoding='utf-8') as file3:
    sentiment_dict = {word.split(',')[0] for word in file3.readlines()}

In [4]:
print('Всего найдено ', len(new_sent_words), ' слов')
print('Пересечение со словарем оценочных выражений: ', len(sentiment_dict & new_sent_words), 'слов, ',
      round(len(sentiment_dict & new_sent_words)/len(new_sent_words), 1)*100, '% списка')

Всего найдено  168  слов
Пересечение со словарем оценочных выражений:  69 слов,  40.0 % списка


In [5]:
for word in sentiment_dict & new_sent_words:
    print(word)

душевный
умеренный
стильный
красивый
улыбчивый
приятный
плохой
тихий
прекрасный
оригинальный
достойный
грубить
предупредительный
осадок
потрясать
отменный
восторженный
классный
превосходный
посредственный
двойственный
дешевый
умница
вкусный
разумный
симпатичный
благодарность
изумительный
приветливый
улыбаться
неплохой
признательность
обширный
благодарный
отличный
дружелюбный
правильный
удивительный
испортиться
демократичный
бедненький
средний
хороший
внимательный
шикарный
обходительный
восхитительный
отвратительный
гуманный
уютный
великолепный
позитивный
негативный
орать
доброжелательный
замечательный
ужасный
живой
положительный
спокойный
качественный
чудесный
противоречивый
приличный
милый
отрицательный
потрясающий
нормальный
вежливый


In [6]:
for word in (new_sent_words - sentiment_dict):
    print(word)

ребята
окраина
спасибо
бегать
девочка
Вю
обслуживание
бейджик
кавказский
молодец
витражный
вкусно
организованный
клиент
атмосферный
официанка
подготовка
ненавязчивый
оформлять
выражать
соня
китайский
периодически
девушка
врубать
выступать
парень
администратор
благодарить
обычный
итальянский
хромать
топтаться
официант
четко
уединенный
музыка
петь
вежливо
громкий
супер
кофейня
объяснять
обалденный
quot оставлять
мексикан
играть
320р
басс
бармен
японский
персонал
купчино
хата
гауд
умничек
фоновый
менеджер
пепперони
суша бар
обслуживать
ачма
обстоятельно
доброжелательно
бейдж
отвечать
мероприятие
разнообразный
громко
манта
впечатление
разнообразие
коллектив
официантка
айс
сервис
стараться
эмоция
мильфей
приемлимый
пытаться
марципановый
сытно
грузинский
хостесса
узбекский
поблагодарить
тетка quot
выключать
разнообразно
подстреливать
свадебный
изменяться
вопрос
задавать
мальчик
разговаривать
воспоминание
управляющий


In [7]:
xlWriter = pd.ExcelWriter('sentiment_new.xlsx', engine='xlsxwriter')
workbook = xlWriter.book
new = [('word', list(new_sent_words - sentiment_dict))]
df2 = pd.DataFrame.from_items(new)
df2.to_excel(xlWriter)
xlWriter.save()